## Data exploration

In [19]:
import pandas as pd
from tqdm import tqdm
import numpy as np
news_df = pd.read_csv("data/Sora_LREC2020_biasedsentences.csv")

Seems that each instance consists of an article with sentences and a bias score between 1 and 4 is assigned to the article as a whole, the title, and each sentence.

In [4]:
news_test_sentence = news_df.iloc[0] # visual inspection
print(f"sentence: {news_test_sentence['s0']}", f"score: {news_test_sentence['0']}")
print(f"dataset score range: {news_df['article_bias'].min(),news_df['article_bias'].max()}")
# pd.DataFrame(news_test_sentence)

sentence: [0]: LOUISVILLE - Dan Johnson posted a final message on Facebook to his friends and family on Wednesday afternoon. score: 3
dataset score range: (np.int64(1), np.int64(4))


In [5]:
# model source: https://huggingface.co/cardiffnlp/twitter-roberta-base-2022-
# adapter source: https://huggingface.co/SOUMYADEEPSAR/text_level_bias_roberta-twitter
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from adapters import AutoAdapterModel
tokeniser = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")
model = AutoAdapterModel.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")
adapter = model.load_adapter("SOUMYADEEPSAR/text_level_bias1", set_active=True)
classifier = pipeline('text-classification', model=model, tokenizer=tokeniser) # cuda = 0,1 based on gpu availability

RobertaAdapterModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['hea

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\spiro\LifeHub\~Education\~Leiden\MSc\Text mining\Assignments\venv\Lib\site-packages\adapters\loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dic

In [6]:
def labels_to_binary(label: int):
    return 0 if label == 1 or label == 2 else 1

# comparing bias score obtained from classifier to true bias score for a test sentence from the news dataset
test_sent_pred = classifier(news_test_sentence["s0"])
test_sent_pred = test_sent_pred[-1]['label']
test_sent_y = labels_to_binary(news_test_sentence["0"])
print(f"pred: {test_sent_pred}, y: {test_sent_y}")

pred: 0, y: 1


In [114]:
test = False
def eval_classifier(df, classifier):
    pred = []
    y = []
    count = 0
    for row_idx,row in tqdm(df.iterrows()):
        if test and row_idx < 20:
            continue
        if test and row_idx == 30:
            break
        for sent_idx in range(20):
            try:
                pred.append(classifier(row[f"s{sent_idx}"])[-1]["label"])
            except ValueError:
                # Todo: add if statement checking if nan
                pred.append(0) # seems (?) to be the case that for NaNs, minimum bias is assigned
            y.append(labels_to_binary(row[f"{sent_idx}"]))
    return pred, y
pred, y = eval_classifier(news_df, classifier)

215it [06:24,  1.79s/it]


In [169]:
def calc_accuracy(pred, y):
    pred_copy = [np.int64(label) for label in pred]
    y_copy = [np.int64(label) for label in y]
    count = 0
    for index,label in enumerate(pred_copy):
        if y_copy[index] == label:
            count += 1
    return count/len(pred_copy)
calc_accuracy(pred, y)

0.6458139534883721

## Fine tune classifier and compare to baseline

In [32]:
#preprocess news data
def extract_sentences_and_labels(df):
    sentences_list = []
    labels_list = []
    for _, row in df.iterrows():
        for i in range(0, 20):  # Sentences are named f"s0" to f"s19"
            if type(row[f"s{i}"]) == str: # skipping nan sentences
                sentences_list.append(row[f"s{i}"])
                labels_list.append(row[f"{i}"])
            
    return sentences_list, labels_list

In [33]:
sentences, labels = extract_sentences_and_labels(news_df)
baseline_tokeniser = tokeniser
inputs = baseline_tokeniser(sentences)

In [ ]:
...
from transformers import TrainingArguments
from transformers import Trainer

args = TrainingArguments(
    "baseline-bias-regressor",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = args,
    
)

## Make predictions for X notes data

In [ ]:
...

## Compare classifier predictions to human predictions

In [ ]:
# my idea for how to evaluate:
# 1) we set some guidelines for how we will evaluate notes 
# 2) we individually make labels for one days worth of notes (that's about 200 notes; this would be the train set)
# 3) we calculate out interrater agreement (if it's very bad, we revise our guidelines and re-label)
# 4) we average our labels and use that to further fine-tune the classifier
# 5) we repeat step 2 on a new set of notes (this would be the test set)
# 6) we make predictions using the classifier obtained from step 4
# 7) either we calculate the MSE using our labels as ground truth, or we calculate three interrater agreements (chico-andrew, chico-classifier, andrew-classifier)
# and see if the human-human agreement is better than the human-classifier agreement.
...